# Лекция 2.2: Классификация и логистическая регрессия

## Классификация

Задача классификации заключается в том, чтобы по входным данным $\mathbf{x} \in \mathbb{R}^d$ предсказать метку класса $y$, которая принадлежит конечному множеству категорий, например,
$$
y \in \{0, 1\} \quad \text{(бинарная классификация)},
$$
или
$$
y \in \{1, 2, \dots, K\} \quad \text{(многоклассовая классификация)}.
$$

Цель обучения заключается в построении модели $f: \mathbb{R}^d \to Y$, которая минимизирует некоторую функцию потерь между истинными метками и предсказаниями.

> Для более подробного изучения логистической регрессии, можете ознакомиться с соответствующей главой [учебника](https://education.yandex.ru/handbook/ml/article/linear-models)

### Логистическая регрессия

#### Постановка задачи

Пусть $y \in \{0, 1\}$, а вектор признаков $\mathbf{x} \in \mathbb{R}^d$. Модель логистической регрессии задаётся следующим образом:
$$
P(y=1 \mid \mathbf{x}) = \sigma(z) = \frac{1}{1 + e^{-z}}, \quad \text{где } z = \mathbf{w}^\top \mathbf{x} + b.
$$
Соответственно,
$$
P(y=0 \mid \mathbf{x}) = 1 - \sigma(\mathbf{w}^\top \mathbf{x} + b).
$$

#### Функция правдоподобия и оптимизация

Для определения параметров $\mathbf{w}$ и $b$ используется метод максимального правдоподобия. Правдоподобие всей выборки:
$$
\mathcal{L}(\mathbf{w}, b) = \prod_{i=1}^N \sigma(\mathbf{w}^\top \mathbf{x}_i + b)^{y_i} \left[1 - \sigma(\mathbf{w}^\top \mathbf{x}_i + b)\right]^{1-y_i}.
$$
Переходя к логарифмическому правдоподобию, получаем:
$$
\ell(\mathbf{w}, b) = \sum_{i=1}^N \left[ y_i \ln \sigma(\mathbf{w}^\top \mathbf{x}_i + b) + (1-y_i) \ln \left(1-\sigma(\mathbf{w}^\top \mathbf{x}_i + b)\right) \right].
$$
Оптимизационная задача сводится к поиску параметров:
$$
\max_{\mathbf{w},\,b} \; \ell(\mathbf{w}, b),
$$
или эквивалентно – к минимизации отрицательного логарифмического правдоподобия (кросс-энтропийной функции потерь).

#### Особенности

- **Линейная граница разделения**: Модель определяет гиперплоскость, которая делит пространство на две области.
- **Использование градиентного спуска**: В аналитическом виде задача не решается, поэтому применяются итерационные методы оптимизации (градиентный спуск, метод Ньютона и др.).

<img src="../../images/log_reg.jpg" alt="Описание изображения" width="600">

#### Статистические соображения

Сигмоида (логистическая функция) определяется как:
$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

Она отображает любое действительное число $z \in \mathbb{R}$ в интервал (0, 1), что удобно для **интерпретации** результата как вероятности.

Производная сигмоиды:
$$
\sigma'(z) = \sigma(z)\,(1-\sigma(z))
$$

Эта форма упрощает вычисления при обучении моделей (например, в алгоритме обратного распространения ошибки).

При моделировании бинарных данных (например, успех/неудача) предполагается, что вероятность $p$ наступления события связана с набором признаков $x$ через линейный предиктор $z$:
$$
z = \mathbf{w}^\top \mathbf{x}.
$$

Для удобства работы с вероятностями вводят **отношение шансов** (odds):
$$
\text{odds} = \frac{p}{1-p}.
$$

Ключевое предположение логит-модели заключается в том, что логарифм отношения шансов линейно зависит от признаков:

$$
\ln\frac{p}{1-p} = z.
$$

Применив экспоненту к обеим частям уравнения, получим:
$$
\frac{p}{1-p} = e^{z}.
$$

Решим это уравнение относительно $p$:
$$
p = (1-p)e^{z} \quad \Longrightarrow \quad p(1 + e^{z}) = e^{z}.
$$

Выразим $p$:
$$
p = \frac{e^{z}}{1 + e^{z}} = \frac{1}{1 + e^{-z}}.
$$

Таким образом, логистическая функция $\sigma(z) = \frac{1}{1+e^{-z}}$ естественным образом появляется как обратное преобразование логита.

Практический пример:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def plot_decision_boundary(model, X, y, ax=None, cmap="coolwarm", h=0.02):
    """
    Строит график разделяющей границы для модели классификации.

    Аргументы:
        model (sklearn-модель): обученная модель классификации.
        X (np.ndarray): массив признаков, размер (n_samples, 2).
        y (np.ndarray): массив целевых меток.
        ax (matplotlib.axes.Axes, optional): ось для построения графика.
        cmap (str): название цветовой карты.
        h (float): шаг для создания сетки.
        
    Возвращает:
        ax (matplotlib.axes.Axes): ось с графиком разделяющей границы.
    """
    if ax is None:
        ax = plt.gca()
    
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    ax.contourf(xx, yy, Z, alpha=0.3, cmap=cmap)
    ax.scatter(X[:, 0], X[:, 1], c=y, edgecolors="k", cmap=cmap)
    
    ax.set_xlabel("Признак 1")
    ax.set_ylabel("Признак 2")
    ax.set_title("Разделяющая граница логистической регрессии")
    
    return ax

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, 
                             confusion_matrix, 
                             classification_report)

In [ ]:
np.random.seed(42)
X, y = make_classification(n_samples=50,
                            n_features=2,
                            n_redundant=0,
                            n_informative=2,
                            n_clusters_per_class=1,
                            flip_y=0.1,
                            class_sep=1.5,
                            random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, 
                                                    random_state=42)

In [ ]:
log_reg = LogisticRegression(solver="lbfgs")
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("Report:\n", classification_report(y_test, y_pred))

plt.figure(figsize=(8, 6))
plot_decision_boundary(log_reg, X, y)
plt.tight_layout()
plt.show()

In [ ]:
print(log_reg.predict_proba(X_test))